# Import Data

In [74]:
# Import libraries

import pandas as pd
from sklearn import linear_model, metrics
import numpy as np
%pylab inline

from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures, StandardScaler

from sklearn.metrics import mean_squared_error, r2_score


warnings.filterwarnings("ignore")


Populating the interactive namespace from numpy and matplotlib


In [75]:
# Import dataset
data = pd.read_csv("/Users/marinejacquemin/Desktop/airline.csv")

In [76]:
data = data.drop(['Unnamed: 0'],1)

In [6]:
#create a dictionary with full airlines names
airlines_names = pd.read_csv('/Users/marinejacquemin/Desktop/airlines.csv',sep=";")
abbr_companies = airlines_names.set_index('IATA_CODE')['AIRLINE'].to_dict()

In [16]:
data.columns

Index(['MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'AIRLINE_ID', 'CARRIER',
       'ORIGIN_AIRPORT_ID', 'ORIGIN', 'ORIGIN_CITY_NAME', 'ORIGIN_STATE_ABR',
       'ORIGIN_STATE_NM', 'DEST_AIRPORT_ID', 'DEST', 'DEST_CITY_NAME',
       'DEST_STATE_ABR', 'DEST_STATE_NM', 'DEP_DELAY', 'DEP_TIME_BLK',
       'ARR_DELAY', 'ARR_TIME_BLK', 'AIR_TIME', 'DISTANCE'],
      dtype='object')

In [8]:
# remove extreme values before fitting
#data = data[data.DEP_DELAY < 60]

In [85]:
data.shape

(5556190, 21)

# Machine Learning part

Before we start, let's define X as the matrix contraining the quantitative features we need for further analysis.
Y is the target vector, corresponding here to the Arrival Delay

In [77]:
# X contains the quantitative features

list_cat_var = ['AIRLINE_ID', 'CARRIER', 'ORIGIN_AIRPORT_ID', 'ORIGIN',
       'ORIGIN_CITY_NAME', 'ORIGIN_STATE_ABR', 'ORIGIN_STATE_NM',
       'DEST_AIRPORT_ID', 'DEST', 'DEST_CITY_NAME', 'DEST_STATE_ABR',
       'DEST_STATE_NM']

X = data.drop([x for x in list_cat_var],1)
X = X.drop(['ARR_DELAY','DEP_DELAY'],1)
X = X.fillna(0)

#Y is the target (arrival delay)
Y = data.ARR_DELAY


In [87]:
X.columns

Index(['MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'DEP_TIME_BLK', 'ARR_TIME_BLK',
       'AIR_TIME', 'DISTANCE'],
      dtype='object')

In [78]:
#Standardize the features
std_scale = StandardScaler().fit(X)
X_scaled = std_scale.transform(X)

In [79]:
#Let's split the data into training and testing sets. 

from sklearn.model_selection import train_test_split

xtrain, xtest, ytrain, ytest = train_test_split(X_scaled, Y, train_size=0.7)

# Multiple Linear Regression

Let's first create the linear regression model

In [2]:
lr = linear_model.LinearRegression()

# We train the model on the training set
lr.fit(xtrain,ytrain)

NameError: name 'linear_model' is not defined

In [ ]:
#Let's compute the model performace

# On récupère l'erreur de norme 2 sur le jeu de données test comme baseline
baseline_error = np.mean((lr.predict(xtest) - ytest) ** 2)

print(baseline_error)

In [ ]:
#On  crée le modèle de régression linéaire et on calcule les performances de cette régression

lm = linear_model.LinearRegression()
model = lm.fit(xtrain,ytrain)
predictions = lm.predict(xtrain)
print("MSE =", metrics.mean_squared_error(predictions, ytrain))

In [ ]:
#number of predictions where the differences with real values is greater than 15 minutes
icount = 0
for i, val in enumerate(ytrain):
    if abs(val-predictions[i]) > 15: icount += 1
'{:.2f}%'.format(icount / len(predictions) * 100)

In [ ]:
model = lm.fit(xtest,ytest)
predictions = lm.predict(xtest)
print("MSE =", metrics.mean_squared_error(predictions, ytest))


In [ ]:
'Ecart = {:.2f} min'.format(np.sqrt(metrics.mean_squared_error(predictions, ytest)))

# Regression polynomiale - ordre 2

In [ ]:
#test avec ordre 2
poly = PolynomialFeatures(degree = 2)
regr = linear_model.LinearRegression()
X_ = poly.fit_transform(xtrain)
regr.fit(X_, ytrain)

In [ ]:
result = regr.predict(X_)
print("MSE =", metrics.mean_squared_error(result, ytrain))

In [ ]:
#calcule du pourcentage d'écart >15min
icount = 0
for i, val in enumerate(ytrain):
    if abs(val-result[i]) > 15: icount += 1
'{:.2f}%'.format(icount / len(result) * 100)

In [ ]:
X_ = poly.fit_transform(xtest)
result = regr.predict(X_)
score = metrics.mean_squared_error(result, ytest)
print("Mean squared error = ", score)

In [ ]:
'Ecart = {:.2f} min'.format(np.sqrt(score))

# Ridge Regression

In [ ]:
#fit on the training set

ridgereg = Ridge(alpha=0.3,normalize=True)
poly = PolynomialFeatures(degree = 2)
X_ = poly.fit_transform(xtrain)
ridgereg.fit(X_, ytrain)

In [ ]:
#testing on the test set

X_ = poly.fit_transform(xtest)
result = ridgereg.predict(X_)
score = metrics.mean_squared_error(result, ytest)
print("Mean squared error = ", score)

In [ ]:
#to determine the best model, we have two free parameters to adjust: the polynomial order and the  α  coefficient of the 'Ridge Regression' 

score_min = 10000
for pol_order in range(2, 3):
    for alpha in range(0, 10, 2):
        ridgereg = Ridge(alpha = alpha/10, normalize=True)
        poly = PolynomialFeatures(degree = pol_order)
        regr = linear_model.LinearRegression()
        X_ = poly.fit_transform(xtrain)
        ridgereg.fit(X_, ytrain)        
        X_ = poly.fit_transform(xtest)
        result = ridgereg.predict(X_)
        score = metrics.mean_squared_error(result, ytest)        
        if score < score_min:
            score_min = score
            parameters = [alpha/10, pol_order]
        print("n={} alpha={} , MSE = {:<0.5}".format(pol_order, alpha, score))

In [ ]:
#on calcule la MSE avec les paramètres à l'optimum : 

ridgereg = Ridge(alpha = parameters[0], normalize=True)
poly = PolynomialFeatures(degree = parameters[1])
X_ = poly.fit_transform(X)
ridgereg.fit(X_, Y)
result = ridgereg.predict(X_)
score = metrics.mean_squared_error(result, Y)        
print(score)

In [ ]:
#average error delay of : 
'Ecart = {:.2f} min'.format(np.sqrt(score))

In [ ]:
r2_score_ridge = r2_score(Y, result)
print("r^2 on test data : %f" % r2_score_ridge)


In [ ]:
# using the default CV
alphas = [0.1, 1, 10, 100, 1e3, 1e4, 2e4, 5e4, 8e4, 1e5, 1e6, 1e7, 1e8]
reg = linear_model.RidgeCV(alphas=alphas, store_cv_values=True)
#reg.fit(train_x, train_y, sample_weight=sample_weight)
reg.fit(xtrain, ytrain)
cv_mse = np.mean(reg.cv_values_, axis=0)
print("alphas: %s" % alphas)
print("CV MSE: %s" % cv_mse)
print("Best alpha using built-in RidgeCV: %f" % reg.alpha_)

In [ ]:
#sous-forme graphique
plt.plot(alphas,cv_mse)
plt.xlabel('Alpha')
plt.ylabel('MSE')
plt.title('MSE vs. Alpha')
plt.show()

# Lasso Regression

In [ ]:
regLasso1 = Lasso(fit_intercept=False,normalize=False)
print(regLasso1)

In [ ]:
#training
regLasso1.fit(xtrain,ytrain)
#compute coefficients
print(regLasso1.coef_)

In [ ]:
#look for optimal model
#lasso path (10 valeurs de alpha à tester)

my_alphas = numpy.array([0,0.001,0.01,0.05,0.1,0.2,0.3,0.4,0.5,1])

In [ ]:
from sklearn.linear_model import lasso_path
alpha_for_path, coefs_lasso, _ = lasso_path(xtrain,ytrain,alphas=my_alphas)

In [ ]:
import matplotlib.cm as cm
couleurs = cm.rainbow(numpy.linspace(0,1,16))
#graphique lasso path (une courbe par variable)
for i in range(coefs_lasso.shape[0]):
    plt.plot(alpha_for_path,coefs_lasso[i,:],c=couleurs[i])
plt.xlabel('Alpha')
plt.ylabel('Coefficients')
plt.title('Lasso path')
plt.show()

In [ ]:
#nombre de coefs. non-nuls pour chaque alpha
nbNonZero = numpy.apply_along_axis(func1d=numpy.count_nonzero,arr=coefs_lasso,axis=0)
print(pd.DataFrame({'alpha':alpha_for_path,'Nb non-zero coefs':nbNonZero}))

In [ ]:
#ou sous forme graphique
plt.plot(alpha_for_path,nbNonZero) 
plt.xlabel('Alpha')
plt.ylabel('Nb. de variables')
plt.title('Nb. variables vs. Alpha')
plt.show()

In [ ]:
#look for optimal solution

from sklearn.linear_model import LassoCV
lcv = LassoCV(alphas=my_alphas,normalize=False,fit_intercept=False,random_state=0,cv=5)
#lancement sur l'échantillon d'apprentissage
lcv.fit(xtrain,ytrain)

#valeurs des alphas qui ont été testés
print(lcv.alphas_)

In [ ]:
#moyenne mse en validation croisée pour chaque alpha
avg_mse = numpy.mean(lcv.mse_path_,axis=1)
#alphas vs. MSE en cross-validation
print(pd.DataFrame({'alpha':lcv.alphas_,'MSE':avg_mse}))

In [ ]:
#sous-forme graphique
plt.plot(lcv.alphas_,avg_mse)
plt.xlabel('Alpha')
plt.ylabel('MSE')
plt.title('MSE vs. Alpha')
plt.show()

In [ ]:
#best alpha
print(lcv.alpha_)

In [ ]:
#prediction with this model
ypLasso = lcv.predict(xtest)
scorelasso = mean_squared_error(ytest,ypLasso)
print(scorelasso)

#résultats moins bon que les précédentes méthodes étudiées

In [ ]:
#average error delay of : 
'Ecart = {:.2f} min'.format(np.sqrt(scorelasso))

In [ ]:
r2_score_lasso = r2_score(ytest, ypLasso)
print("r^2 on test data : %f" % r2_score_lasso)

# Elastic net Regression

In [ ]:
enet = ElasticNet(alpha=alpha, l1_ratio=0.7)

y_pred_enet = enet.fit(xtrain, ytrain).predict(xtest)
r2_score_enet = r2_score(ytest, y_pred_enet)
print(enet)
print("r^2 on test data : %f" % r2_score_enet)

In [ ]:
#Look for optimal model

from sklearn.linear_model import enet_path

my_alphas_e = numpy.array([0.001,0.01,0.05,0.1,0.2,0.3,0.4,0.5,1,2,3])
alpha_for_path, coefs_enet, _ = enet_path(xtrain,ytrain,alphas=my_alphas_e)

In [ ]:
import matplotlib.cm as cm
couleurs = cm.rainbow(numpy.linspace(0,1,16))
#graphique elastic net path (une courbe par variable)
for i in range(coefs_enet.shape[0]):
    plt.plot(alpha_for_path,coefs_enet[i,:],c=couleurs[i])
plt.xlabel('Alpha')
plt.ylabel('Coefficients')
plt.title('Enet path')
plt.show()

In [ ]:
#nombre de coefs. non-nuls pour chaque alpha
nbNonZero = numpy.apply_along_axis(func1d=numpy.count_nonzero,arr=coefs_enet,axis=0)
print(pd.DataFrame({'alpha':alpha_for_path,'Nb non-zero coefs':nbNonZero}))

In [ ]:
#ou sous forme graphique
plt.plot(alpha_for_path,nbNonZero) 
plt.xlabel('Alpha')
plt.ylabel('Nb. de variables')
plt.title('Nb. variables vs. Alpha')
plt.show()

In [ ]:
#recherche de la solution la plus performante

from sklearn.linear_model import ElasticNetCV
ecv = ElasticNetCV(alphas=my_alphas_e,normalize=False,fit_intercept=False,random_state=0,cv=5)
#lancement sur l'échantillon d'apprentissage
ecv.fit(xtrain,ytrain)

In [ ]:
#moyenne mse en validation croisée pour chaque alpha
avg_mse = numpy.mean(ecv.mse_path_,axis=1)
#alphas vs. MSE en cross-validation
print(pd.DataFrame({'alpha':ecv.alphas_,'MSE':avg_mse}))

In [ ]:
#prediction avec ce modèle
ypEnet = ecv.predict(xtest)

print(mean_squared_error(ytest,ypEnet))

In [ ]:
y_pred_enet = ecv.fit(xtrain, ytrain).predict(xtest)
r2_score_enet = r2_score(ytest, y_pred_enet)
print("r^2 on test data : %f" % r2_score_enet)

# Random Forest Regressor

In [ ]:
# X contient les features quantitatives dont on a besoin

list_cat_var = ['AIRLINE_ID','ORIGIN','DEST','CARRIER',
       'ORIGIN_CITY_NAME', 'ORIGIN_STATE_ABR', 'ORIGIN_STATE_NM', 'DEST_CITY_NAME', 'DEST_STATE_ABR',
       'DEST_STATE_NM']

#data = data.fillna(0)
data["MONTH"] = data["MONTH"].astype('category')
data["DAY_OF_WEEK"] = data["DAY_OF_WEEK"].astype('category')
data["DAY_OF_MONTH"] = data["DAY_OF_MONTH"].astype('category')
data["ORIGIN_AIRPORT_ID"] = data["ORIGIN_AIRPORT_ID"].astype('category')
data["DEST_AIRPORT_ID"] = data["DEST_AIRPORT_ID"].astype('category')

#on filtre sur une compagnie, pour réduire le temps d'éxcution
Xrf = data.loc[data['CARRIER'] == "AA"].drop([x for x in list_cat_var],1)
Xrf = Xrf.drop(['ARR_DELAY','DEP_DELAY'],1)

#Y est la target (le retard à l'arrivée)
Yrf = data.loc[data['CARRIER'] == "AA"].ARR_DELAY

In [ ]:
#on standardise les features
std_scale = StandardScaler().fit(Xrf)
X_scaled = std_scale.transform(Xrf)

In [ ]:
#on sépare le jeu de données entre training et testing sets
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(Xrf, Yrf, train_size=0.7)

In [ ]:
# définition des paramètres
forest = RandomForestRegressor(n_estimators=20, max_depth=10,
min_samples_split=2, min_samples_leaf=1,
   max_features='auto', max_leaf_nodes=None,
   bootstrap=True, oob_score=True)

# apprentissage
forest = forest.fit(x_train,y_train)
print(1-forest.oob_score_)

# erreur de prévision sur le test
1-forest.score(x_test,y_test)

In [ ]:
#Optimisation du paramètre max_features par validation croisée.

from sklearn.grid_search import GridSearchCV

param=[{"max_features":list(range(2,8))}]
delay_rf= GridSearchCV(RandomForestRegressor(
n_estimators=15,max_depth=10),param,cv=5,n_jobs=-1)
delay_rf=delay_rf.fit(x_train, y_train)

# paramètre optimal
param = delay_rf.best_params_

In [ ]:
#utilisation de la valeur optimale
￼forest = RandomForestRegressor(n_estimators=20, max_depth=10,
   min_samples_split=2, min_samples_leaf=1,max_features=param)

# apprentissage
forest = forest.fit(x_train,y_train)

predictions = forest.predict(x_test)

# Calculate the absolute errors
errors = abs(predictions - y_test)

# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2),"min")

score_rf = mean_squared_error(y_test,predictions)
print(score_rf)

r2_score_rf = r2_score(y_test, predictions)
print("r^2 on test data : %f" % r2_score_rf)

#average error delay of : 
'Ecart = {:.2f} min'.format(np.sqrt(score_rf))